In [ ]:
import tensorflow as tf
import numpy as np
import os
from performance_prediction_model import Model
from sklearn.externals import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
import prediction

In [ ]:
model = 'all'          # Model
folder = 'DGX1_V100'   # Data folder
path = 'data/other'

In [ ]:
# Select optimizer ('None', 'SGD', 'Adam', ...)
opt = 'None'

In [ ]:
batchsize = 2**np.arange(0,6,1)
t = np.zeros([16,6])
for i in range(len(batchsize)):
    l, t[:,i] = prediction.predict_walltime('model/architecture/VGG16.json',
                                            'models/%s/saved_model' %model,
                                            'models/%s/scaler_Conv.save' %model, 
                                            batchsize[i],
                                            opt,
                                            900,
                                            5120,
                                            1455)

In [ ]:
l_unique = l.copy()
duplicates = [12,11,9,6]
for d in duplicates:
    print(l_unique[d])
    l_unique.pop(d)
    
t_unique = np.delete(t,duplicates,0)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[8,8])
plt.plot(batchsize,t_unique.transpose(),'o-')

ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.xaxis.set_minor_formatter(plt.NullFormatter())
ax.yaxis.set_minor_formatter(plt.NullFormatter())

matplotlib.rc('xtick', labelsize=25) 
matplotlib.rc('ytick', labelsize=25) 

plt.xlabel('batch size',fontsize=25)
plt.ylabel('predicted time (ms)',fontsize=25)

plt.legend(l_unique,bbox_to_anchor=(1.04,1), loc="upper left",fontsize=20)

plt.tight_layout()

plt.savefig('figures/VGG16_full_model_layers_%s_%s.png' %(model,opt), dpi=300)
plt.show()

In [ ]:
np.savetxt('/Users/djustus/layer_vs_batchsize_%s_%s.csv' %(model,opt),t)

In [ ]:
sum(t)

In [ ]:
benchmark = pd.read_csv('%s/benchmark_VGG16_V100_fw_20181022.csv' %(path))
benchmark = benchmark.drop(6)

In [ ]:
benchmark['prediction'] = sum(t)

In [ ]:
benchmark.columns = ['operation', 'imsize', 'precision', 'batchsize',
       'time_batch', 'performance', 'memory',
       'comment', 'prediction']

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[8,8])
benchmark.plot('batchsize',['time_batch','prediction'],ax=ax)
# ax.set_yscale('log')
# ax.set_xscale('log')
plt.show()

In [ ]:
benchmark